In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from PIL import Image
import os
import string
import matplotlib.pyplot as plt



In [2]:
##we need to switch up the training set

train=pd.read_csv('/content/drive/MyDrive/Datasets/sign_mnist_train.csv')

test=pd.read_csv('/content/drive/MyDrive/Datasets/sign_mnist_test.csv')


In [15]:
##getting the training data and training labels

##we one hot encoded the labels

def load_data(path):
    df = pd.read_csv(path)
    y = np.array([label if label < 9
                  else label-1 for label in df['label']])
    df = df.drop('label', axis=1)
    x = np.array([df.iloc[i].to_numpy().reshape((28, 28))
                  for i in range(len(df))]).astype(float)
    x = np.expand_dims(x, axis=3)
    y = pd.get_dummies(y).values

    return x, y

X_train, Y_train = load_data('/content/drive/MyDrive/Datasets/sign_mnist_train.csv')
X_test, Y_test = load_data('/content/drive/MyDrive/Datasets/sign_mnist_test.csv')

In [4]:
X_train[0].shape

(28, 28, 1)

In [5]:
train_labels=train.loc[:,"label"]
test_labels=test.loc[:,'label']

train=train.drop(labels='label',axis=1).to_numpy()
test=test.drop(labels='label',axis=1).to_numpy()



In [6]:
##reshaping training andtesting arrays

def shape(example):
  return example.reshape(28,28,1)

train=np.array(list(map(shape,train)))
test=np.array(list(map(shape,test)))


In [38]:
Y_train.shape, train_encoded.shape

((27455, 24), (27455, 25))

In [7]:
##one hot endocidn the training and testing_labels

train_encoded = np.zeros((train_labels.size, train_labels.max() + 1))
train_encoded[np.arange(train_labels.size), train_labels] = 1

test_encoded = np.zeros((test_labels.size, test_labels.max() + 1))
test_encoded[np.arange(test_labels.size), test_labels] = 1

In [31]:
##time to build the model

##labelst ot unint8 and training to float65

train=train.astype(np.float64)
test=test.astype(np.float64)

test_encoded=test_encoded.astype(np.uint8)
train_encoded=train_encoded.astype(np.uint8)

In [107]:
##creating the neural network
import tensorflow as tf
import keras


model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=32,
                           kernel_size=(3, 3),
                           activation='relu',
                           input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(filters=64,
                           kernel_size=(3, 3),
                           activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(25, activation='softmax')
])

In [108]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [126]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 13, 13, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 5, 5, 64)         0         
 g2D)                                                            
                                                                 
 flatten_6 (Flatten)         (None, 1600)              0         
                                                                 
 batch_normalization_12 (Bat  (None, 1600)            

In [109]:
model.fit(train, train_encoded, epochs=5, validation_data=(test, test_encoded))

##convert the encodings th dtypunit8

Epoch 1/5
858/858 [==============================] - 39s 43ms/step - loss: 0.2408 - accuracy: 0.9400 - val_loss: 0.4310 - val_accuracy: 0.8788
Epoch 2/5
858/858 [==============================] - 35s 41ms/step - loss: 0.0171 - accuracy: 0.9979 - val_loss: 2.5152 - val_accuracy: 0.5102
Epoch 3/5
858/858 [==============================] - 37s 43ms/step - loss: 0.0505 - accuracy: 0.9856 - val_loss: 0.2562 - val_accuracy: 0.9456
Epoch 4/5
858/858 [==============================] - 36s 41ms/step - loss: 0.0030 - accuracy: 0.9999 - val_loss: 0.2723 - val_accuracy: 0.9331
Epoch 5/5
858/858 [==============================] - 34s 40ms/step - loss: 0.0394 - accuracy: 0.9886 - val_loss: 0.4376 - val_accuracy: 0.9117


In [110]:
test_loss, test_acc = model.evaluate(test, test_encoded)
print(f"Test accuracy: {test_acc}")

225/225 [==============================] - 3s 14ms/step - loss: 0.4376 - accuracy: 0.9117
Test accuracy: 0.9117401242256165


In [115]:
model.save('/content/models/sign_model')

In [141]:
img = tf.keras.utils.load_img(
    '/content/drive/MyDrive/Datasets/A.jpeg', target_size=(28, 28)
)

img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

gray_img_array = np.mean(img_array, axis=-1, keepdims=True)

gray_img_array.shape

predictions = model.predict(gray_img_array)
score = tf.nn.softmax(predictions[0])

predicted_class = np.argmax(predictions)

predicted_class

1/1 [==============================] - 0s 39ms/step


6